In [79]:
import requests
from collections import defaultdict
import networkx as nx

In [14]:
def fetch_transactions(
    address="vitalik.eth", 
    start_block=0, 
    end_block=99999999, 
    api_key="key",
):
    """Fetch all transactions for a specific Ethereum address using the Etherscan API."""
    # url = "https://api.etherscan.io/api"
    url = "https://api-sepolia.etherscan.io/api"
    params = {
        "module": "account",
        "action": "txlist",
        "address": address,
        "startblock": start_block,
        "endblock": end_block,
        "sort": "asc",  # Sort by ascending order
        "apikey": api_key
    }

    response = requests.get(url, params=params)
    data = response.json()

    # Check for errors in the response
    if data["status"] != "1":
        raise Exception(f"Error: {data['message']} - {data.get('result', '')}")

    return data["result"]

In [42]:
def filter_transactions(tx, direction="send"):
    if (direction == "receive" and tx['from'].lower() == address.lower()) \
    or (direction == "send" and tx['to'].lower() == address.lower()):
        return tx
    return False

In [17]:
address = "0x2461214FC9777705b962248104a58f52BF41B3db"
api_key = "5G6782ZVFEXITQJ86SXA3PTBWRUZGQ74Q3"

In [62]:
def ptx(tx):
    def direction():
        if tx['to'].lower() == address.lower():
            return "Received: "
        elif tx['from'].lower() == address.lower():
            return "Send    : "
        else:
            return "Value   : "
    print(
        "Hash: {}...{}, Block: {}, To: {}...{}, From: {}...{}, {}: {} ETH".format(
            tx['hash'][:4],
            tx['hash'][-3:],
            
            tx['blockNumber'],
            
            tx['to'][:4],
            tx['to'][-3:],
            
            tx['from'][:4],
            tx['from'][-3:],
            
            direction(),
            
            int(tx['value']) / 1e18,
        )
    )

In [63]:
try:
    transactions = fetch_transactions(address=address, start_block=0, end_block=99999999, api_key=api_key)
    print(f"Found {len(transactions)} transactions:")
    
    for tx in transactions:  # Display first 5 transactions for brevity
        ptx(tx)
except Exception as e:
    print(e)

Found 6 transactions:
Hash: 0xb6...07f, Block: 7302265, To: 0x24...3db, From: 0x28...835, Received: : 0.2 ETH
Hash: 0x88...bef, Block: 7302271, To: 0x7d...93d, From: 0x24...3db, Send    : : 0.1 ETH
Hash: 0xde...79f, Block: 7302273, To: 0xf8...a39, From: 0x24...3db, Send    : : 0.04 ETH
Hash: 0x6a...05d, Block: 7302273, To: 0x4e...dd8, From: 0x24...3db, Send    : : 0.04 ETH
Hash: 0x78...338, Block: 7302292, To: 0x24...3db, From: 0x73...29c, Received: : 0.01 ETH
Hash: 0x67...e7c, Block: 7302294, To: 0x24...3db, From: 0xf8...a39, Received: : 0.01 ETH


We need to set a maximum depth
We also need to not loop, example:
- A send to B, B send to C, C send to A

We should diferenciate 3 different end of branch:
- Reach a dead end (no out tx, or only tx that would loop)
- Reach max depth
- Loop to any sender from the branch

In [ ]:
def verbose_address(address):
    return f"{address[:4]}...{address[-3:]}"

In [ ]:
def build_graph(graph, address, direction, max_depth, current_depth=0, visited=set()):
    print("Calling build_graph")
    if current_depth >= max_depth or address in visited:
        print("Max depth or address visited")
        return  # End branch

    # visited.add(address)  # Mark the address as visited

    # Fetch transactions for this address
    txs = fetch_transactions(address, api_key=api_key)
    # print(f"Found {len(txs)} txs visiting address {address[:4]}...{address[-3:]}")

    # If no transactions, end branch
    if not txs:
        print("No tx")
        return

    for tx in txs:
        ptx(tx)
        # if filtered_tx := filter_transactions(tx):
        receiver = tx["to"]  # Adjust based on direction
        if receiver != address:
            # graph[address].append(receiver)
            graph.add_edge(verbose_address(address), verbose_address(receiver))
        else:
            visited.add(address)  # Mark the address as visited
        # Recursively build the graph for the receiver
        build_graph(graph, receiver, direction, max_depth, current_depth + 1, visited)

In [25]:
# Initialize graph
graph = defaultdict(list)

# Start building the graph
build_graph(address, "send", 3)

In [ ]:
G = nx.DiGraph()  # Directed graph
build_graph(G, address, "send", 5)

In [ ]:
nx.draw(G, with_labels=True)